<a href="https://colab.research.google.com/github/NeerajP2003/118S--Agent-AI/blob/main/Coding_Exercise_Prompt_Engineering_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# self_reflection_summary_demo.py
# Goal: Critique summary of the prompt and keep improving it over time (simulated, no external APIs).

from typing import List, Tuple

# ---------------------------
# Prompts (for documentation)
# ---------------------------
PROMPT_INITIAL_SUMMARY = """Summarize the following article in 5–7 concise sentences for a general audience.
Text: {text}
"""

PROMPT_CRITIQUE = """Critique the summary. Check for clarity, missing key points, verbosity, and bias.
List 3–5 actionable improvements.
"""

PROMPT_IMPROVE = """Rewrite the summary incorporating the improvements. Keep it 4–6 sentences,
use plain language, and include one explicit limitation of the summary.
"""

# -------------------------------------
# Tiny “heuristics” to simulate an LLM
# -------------------------------------
def naive_split_sentences(text: str) -> List[str]:
    # Minimal sentence splitter (naive but dependency-free)
    import re
    # Split on ., !, ? and keep texty bits
    chunks = re.split(r'(?<=[\.!?])\s+', text.strip())
    return [c.strip() for c in chunks if c.strip()]

def initial_summary(text: str) -> str:
    """
    Very simple extractive/abstractive hybrid:
    - Grab a few leading sentences as the "gist"
    - Rephrase lightly to sound like a high-level summary
    """
    sents = naive_split_sentences(text)
    if not sents:
        return "The article provides information, but the text is empty."

    # Take up to 4 sentences as the base
    picked = sents[:4]
    base = " ".join(picked)

    # Light framing to make it feel like a summary
    summary = (
        "This article discusses the main goals, actions taken, and outcomes. "
        + base
    )
    # Trim to ~5–7 sentences by adding a concluding line if needed
    if len(naive_split_sentences(summary)) < 5:
        summary += " It concludes with results and remaining challenges."
    return summary

def critique_summary(original_text: str, summary: str) -> List[str]:
    """
    Produce 3–5 actionable critique points:
    - Check for presence of metrics/quantification
    - Mention any missing challenges/limitations
    - Clarity/verbosity
    - Bias/balance
    """
    notes = []

    # Heuristic checks
    has_numbers = any(ch.isdigit() for ch in summary)
    if not has_numbers:
        notes.append("Add concrete metrics or specific facts where available.")

    if "challenge" not in summary.lower() and "limitation" not in summary.lower():
        notes.append("Explicitly mention challenges or limitations to balance the tone.")

    # Clarity/verbosity: suggest tightening
    if len(summary.split()) > 120:
        notes.append("Tighten wording; remove repetition to keep it concise (under ~100 words).")
    else:
        notes.append("Improve flow with smoother topic transitions and avoid repetition.")

    # Balance / bias
    if "results" in summary.lower() and "problem" not in summary.lower():
        notes.append("Balance positive results with unresolved problems or trade-offs.")

    # Coverage: ensure tools/approach or context mentioned
    if "how" not in summary.lower() and "approach" not in summary.lower():
        notes.append("Briefly note the approach or key methods used to achieve results.")

    # Keep list to 3–5 items
    return notes[:5]

def improve_summary(original_text: str, summary: str, critique_notes: List[str]) -> str:
    """
    Rewrite the summary applying the critique:
    - Add a concrete metric if we can infer any from the original text
    - Mention one explicit limitation
    - Keep 4–6 sentences
    """
    # Try to skim simple metrics from original text (very naive):
    # Look for patterns like "doubled", "increased X%", "reduced Y%"
    sents = naive_split_sentences(original_text)
    metric_snippets = []
    for s in sents:
        lower = s.lower()
        if any(k in lower for k in ["%", "percent", "doubled", "halved", "reduced", "increase", "decrease", "dropped"]):
            metric_snippets.append(s)
    metric_hint = metric_snippets[0] if metric_snippets else None

    # Build improved summary
    improved = []
    improved.append("The article explains the context, goals, and the changes made, then evaluates the results.")
    # Include a methods/approach sentence if missing
    if any("approach" in n.lower() or "methods" in n.lower() for n in critique_notes):
        improved.append("It outlines the approach at a high level, focusing on the tools and steps used to reach the outcomes.")
    else:
        improved.append("It briefly describes the approach and the steps taken.")

    # Add metric or concrete outcome if we found a hint
    if metric_hint:
        improved.append(f"It provides concrete outcomes—for instance: {metric_hint}")
    else:
        improved.append("It cites concrete outcomes where available, such as changes in key performance metrics.")

    # Add balance / challenges
    if any("challenge" in n.lower() or "trade-off" in n.lower() or "problem" in n.lower() for n in critique_notes):
        improved.append("The discussion balances improvements with trade-offs and open problems.")
    else:
        improved.append("Alongside improvements, it acknowledges remaining trade-offs and open problems.")

    # Add explicit limitation
    improved.append("One limitation of this summary is that it may omit nuanced details and relies on the article’s reported figures without independent verification.")

    # Ensure 4–6 sentences max
    improved = improved[:6]
    return " ".join(improved)

# -------------------------------------
# Demo text (you can replace this)
# -------------------------------------
ARTICLE = (
    "A mid-sized e-commerce company migrated from a monolithic architecture to microservices over nine months. "
    "The goals were to improve deployment frequency, reduce downtime, and enable team autonomy. "
    "They adopted Kubernetes, a service mesh, and centralized observability. "
    "Challenges included service boundaries, data consistency, and developer onboarding. "
    "Post-migration, deployment frequency doubled and MTTR dropped by 40%, but costs rose 18% and cross-service debugging remained difficult."
)

if __name__ == "__main__":
    print("=== Initial Summary Prompt ===\n", PROMPT_INITIAL_SUMMARY.format(text="<omitted for brevity>"))
    s0 = initial_summary(ARTICLE)
    print("\n=== Initial Summary ===\n", s0)

    print("\n=== Self-Reflection / Critique Prompt ===\n", PROMPT_CRITIQUE)
    notes = critique_summary(ARTICLE, s0)
    print("\n=== Critique (3–5 actionable improvements) ===")
    for i, n in enumerate(notes, 1):
        print(f"{i}) {n}")

    print("\n=== Improved Summary Prompt ===\n", PROMPT_IMPROVE)
    s1 = improve_summary(ARTICLE, s0, notes)
    print("\n=== Improved Summary ===\n", s1)

=== Initial Summary Prompt ===
 Summarize the following article in 5–7 concise sentences for a general audience.
Text: <omitted for brevity>


=== Initial Summary ===
 This article discusses the main goals, actions taken, and outcomes. A mid-sized e-commerce company migrated from a monolithic architecture to microservices over nine months. The goals were to improve deployment frequency, reduce downtime, and enable team autonomy. They adopted Kubernetes, a service mesh, and centralized observability. Challenges included service boundaries, data consistency, and developer onboarding.

=== Self-Reflection / Critique Prompt ===
 Critique the summary. Check for clarity, missing key points, verbosity, and bias.
List 3–5 actionable improvements.


=== Critique (3–5 actionable improvements) ===
1) Add concrete metrics or specific facts where available.
2) Improve flow with smoother topic transitions and avoid repetition.
3) Briefly note the approach or key methods used to achieve results.

===